In [2]:
import os
import pandas as pd
import numpy as np
import sys
import csv
from time import strptime
import json
from datetime import datetime as dt
from datetime import date
from statistics import mean
import math
import pickle
import time

In [3]:
# csv.field_size_limit(sys.maxsize)
# np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:20,.15f}'.format
# pd.options.display.max_rows = 10000
# pd.set_option('display.max_colwidth', -1)

In [4]:
def calc_smooth_mean(df, by, on, m):
    mean = df[on].mean()
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']
    smooth = (counts * means + m * mean) / (counts + m)
    return df[by].map(smooth)

In [5]:
df_file = pd.read_csv('savedcsv/17_homepage.csv')
df_file = df_file.drop(df_file[df_file['META__id'] == 6].index)
df_file.to_csv('datasets/df_file.csv', index=False)
df_file = pd.read_csv('datasets/df_file.csv')

In [26]:
# df_raw = reduce_mem_usage(pd.read_csv('savedcsv/17_homepage.csv'))
df_raw = pd.read_csv('savedcsv/17_homepage.csv')
df_raw['META__profitability'] = df_raw['profitability']
df_raw['META__year_avg_profitability'] = df_raw['year_avg_profitability']
df_raw = df_raw.drop(['profitability', 'year_avg_profitability', 'PROCESS__original_language'], axis=1)
drop_cols = [c for c in df_raw.columns if 'META' not in c and ('avg' in c or 'experience' in c or 'movies_before' in c)]
df_raw = df_raw.drop(drop_cols, axis=1)
df_raw = df_raw.drop(df_raw[df_raw['META__id'] == 6].index)
df_raw.to_csv('datasets/df_raw.csv', index=False)
df_raw = pd.read_csv('datasets/df_raw.csv')

In [59]:
df_raw = pd.read_csv('datasets/df_raw.csv')
# for i in range(1, 9):
#     df_raw.rename(columns={f'META__cast_{i}_name': f'META__cast_{i}'}, inplace=True)
# df_raw.rename(columns={
#     'META__crew__editing__editor__1': 'META__crew__editing__editor', 
#     'META__crew__writing__screenplay__1': 'META__crew__writing__screenplay',
#     'META__crew__production__executive_producer__1': 'META__crew__production__executive_producer',
#     'META__crew__directing__director__1': 'META__crew__directing__director'
#     }, inplace=True)
# df_raw.rename(columns={
#     'META__crew__production__producer__1': 'META__crew__production__producer_1',
#     'META__crew__production__producer__2': 'META__crew__production__producer_2'
#     }, inplace=True)
# df_raw.to_csv('datasets/df_raw.csv', index=False)

In [186]:
get_group_avg_revenue = lambda group: np.rint(group['META__revenue'].mean()) if not group.empty else np.nan
get_group_avg_profit = lambda group: np.rint((group['META__revenue'] - group['budget']).mean()) if not group.empty else np.nan


class CalculateInfo:
    def __init__(self, df, movie, movie_index, df_ref):
        self.df_ref = df_ref
        self.df = df
        self.movie = movie
        self.movie_index = movie_index
        self.year, self.month, self.day = movie.META__year, movie.META__month, movie.META__day
        self.movies_before = self._get_movies_before()

    def _get_movies_before(self):
        years, months, days = self.df_ref.META__year, self.df_ref.META__month, self.df_ref.META__day
        return self.df_ref[(years < self.year) | ((years == self.year) & (months < self.month)) | ((years == self.year) & (months == self.month) & (days < self.day))]

    def _get_col_index(self, col):
        if col not in self.df:
            self.df[col] = np.nan
        return self.df.columns.get_loc(col)

    def _set_column(self, column, suffix, value):
        self.df.iat[self.movie_index, self._get_col_index(f'{column}_{suffix}')] = value

    def _set_avg_profit(self, column, group): self._set_column(column, 'avg_profit', get_group_avg_profit(group))
    def _set_avg_revenue(self, column, group): self._set_column(column, 'avg_revenue', get_group_avg_revenue(group))
    def _set_movies_before(self, column, group): self._set_column(column, 'movies_before', group.shape[0])
    def _set_experience(self, column, group): self._set_column(column, 'experience', self.movie.META__year - group['META__year'].min() if not group.empty else 0)

    def _get_groups(self, column, iteration, df_ref):
        def_columns = ['budget', 'META__revenue', 'META__year']
        df = self.movies_before if not df_ref else self.df_ref
        if len(iteration) > 1:
            return pd.concat([df[[f'{column}_{i}'] + def_columns].rename(columns={f'{column}_{i}': column}) for i in iteration]).groupby(column)
        else:
            _df = df[def_columns] if column in def_columns else df[[column] + def_columns]
            return _df.groupby(column)

    def _set_info(self, column_to_iterate, new_column, iteration, df_ref=False,
                  set_profit=False, set_revenue=False, set_budget=False, set_experience=False, set_movies_before=False):
        groups = self._get_groups(column_to_iterate, iteration, df_ref)
        for i in iteration:
            new_column_full = f'{new_column}_{i}' if len(iteration) > 1 else new_column
            value = row[f'{column_to_iterate}_{i}'] if len(iteration) > 1 else row[column_to_iterate]
            group = groups.get_group(value) if value in groups.groups.keys() else pd.DataFrame()
            if set_profit:
                self._set_avg_profit(new_column_full, group)
            if set_revenue:
                self._set_avg_revenue(new_column_full, group)
            if set_experience:
                self._set_experience(new_column_full, group)
            if set_movies_before:
                self._set_movies_before(new_column_full, group)

    def set_info_year(self):
        self._set_info(
            column_to_iterate='META__year', new_column='year', iteration=range(1),
            df_ref=True,
            set_revenue=True, set_profit=True)
        return self

    def set_info_production_company(self):
        self._set_info(
            column_to_iterate='META__production_company', new_column='production_company', iteration=range(1,4),
            set_profit=True, set_revenue=True)
        return self

    def set_info_cast(self):
        self._set_info(
            column_to_iterate='META__cast', new_column='cast', iteration=range(1,9),
            set_profit=True, set_revenue=True, set_experience=True, set_movies_before=True)
        return self

    def set_info_collections(self):
        self._set_info(
            column_to_iterate='META__collection_name', new_column='collection', iteration=range(1),
            set_profit=True, set_revenue=True)
        return self

    def set_info_crew(self):
        crew_columns = [column for column in self.df.columns if 'META__crew' in column]
        producer_passed = False
        for column in crew_columns:
            if 'production__producer' in column and not producer_passed:
                column_to_iterate, new_column, iteration = column[:-2], column[6:-2], range(1,3)
                producer_passed = True
            else:
                column_to_iterate, new_column, iteration = column, column[6:], range(1)
            self._set_info(
                column_to_iterate=column_to_iterate, new_column=new_column, iteration=iteration,
                set_profit=True, set_revenue=True, set_movies_before=True)
        return self


    def set_info_cast_avg(self):
        key_columns = ['avg_revenue', 'avg_profit', 'experience', 'movies_before']
        for key_column in key_columns:
            df_columns = [f'cast_{i}_{key_column}' for i in range(1,9)]
            movie = self.df.iloc[self.movie_index]
            self._set_column('cast_avg', key_column, np.mean([movie[col] for col in df_columns if not pd.isna(movie[col])]))

In [187]:
start = time.time()


dff = df_raw[:10].copy()
for row_index, row in dff.iterrows():
    CalculateInfo(dff, row, row_index, df_ref=df_raw) \
        .set_info_cast() \
        .set_info_collections() \
        .set_info_crew() \
        .set_info_production_company() \
        .set_info_year() \
        .set_info_cast_avg()
print(time.time() - start)

5.999499320983887


In [118]:
df_compare = pd.read_csv('datasets/dataset_all.csv')

In [7]:
df_raw.META__revenue.quantile([0.05, 0.95])

0.050000000000000       111,229.200000000011642
0.950000000000000   348,514,752.599999785423279
Name: META__revenue, dtype: float64

In [8]:
num = df_raw[
    (df_raw.META__profitability > df_raw.META__profitability.quantile(0.01)) & 
    (df_raw.META__profitability < df_raw.META__profitability.quantile(0.99)) &
    (df_raw.META__year >= 2000) &
    (df_raw.META__year <= 2018) &
    (df_raw.budget >= 250000)
].shape[0]

print(num)
print(num/df_raw.shape[0])

4600
0.6139063125583878


In [9]:
num = df_raw[
    (df_raw.META__revenue > df_raw.META__profitability.quantile(0.01)) & 
    (df_raw.META__profitability < df_raw.META__profitability.quantile(0.99)) &
    (df_raw.META__year >= 2000) &
    (df_raw.META__year <= 2018) &
    (df_raw.budget >= 250000)
].shape[0]

print(num)
print(num/df_raw.shape[0])

4660
0.6219137862004538


In [28]:
def get_real_columns(name_prefix, name_suffix, num_columns):
    return [f'{name_prefix}{j}{name_suffix}' for j in range(1, num_columns+1)]

def get_date_from_movie(movie):
    return date(movie['META__year'], movie['META__month'], movie['META__day'])


def get_info_df_movies_per_value_range_columns(df, columns):
    result = {}
    for i, row in df.iterrows():
        for col in columns:
            value = row[col]
            if not pd.isna(value):
                if value in list(result.keys()):
                    result[value]['movies'] = result[value]['movies'].append(df.iloc[[i]])
                else:
                    result[value] = {'movies': df.iloc[[i]]}
    return result

# def get_info_df_movies_per_value_range_columns(df, columns):
#     def rows_with_value(value):
#         return df[df.apply(lambda row: value in [row[column] for column in columns], axis=1)]

#     unique_values = list(set([movie[column] for i, movie in df.iterrows() for column in columns]))
#     return {value: rows_with_value(value) for value in unique_values}

def calculate_info_per_previous_movies_per_value(df, 
                                                 info_dict, 
                                                 calc_budget=False, 
                                                 calc_movies_before=False, 
                                                 calc_experience=False, 
                                                 calc_revenue=False,
                                                ):
    
    def get_average_profit(df):
        return (df['META__revenue'] - df['budget']).mean()
    
    def get_experience(df, row):
        return (row['META__year'] - df.META__year.min())
    
    def is_movie_before(movie, compare_date):
        return date(movie.META__year, movie.META__month, movie.META__day) < compare_date

    for value_name in info_dict:
        value = info_dict[value_name]
        value['avg_profit'] = {}
        if calc_budget:
            value['avg_budget'] = {}
        if calc_experience:
            value['experience'] = {}
        if calc_revenue:
            value['avg_revenue'] = {}
        if calc_movies_before:
            value['movies_before'] = {}

        movies = value['movies']
        for movie_index, movie in movies.iterrows():
            current_date = get_date_from_movie(movie)
            str_date = str(current_date)
            if calc_experience and str_date not in list(value['experience'].keys()):
                value['experience'][str_date] = get_experience(movies, movie)
            if str_date not in list(value['avg_profit'].keys()):
                movies_before = movies[movies.apply(lambda row: is_movie_before(row, current_date), axis=1)]
                if not movies_before.empty:
                    value['avg_profit'][str_date] = get_average_profit(movies_before)
                    if calc_movies_before:
                        value['movies_before'][str_date] = movies_before.shape[0]
                    if calc_revenue:
                        value['avg_revenue'][str_date] = movies_before['META__revenue'].mean()                        
                    if calc_budget:
                        value['avg_budget'][str_date] = movies_before['budget'].mean()
                else:
                    if calc_movies_before:
                        value['movies_before'][str_date] = 0
                    

def get_columns_from_info_dict(df, info_dict, real_columns, new_column_prefix='', nested_info=False):
    def get_first_key(d):
        return d[list(d.keys())[0]]
    
    new_columns_data = {}
    info_names = [key for key in get_first_key(info_dict).keys() if key != 'movies']
    print(info_names)
    for index, real_column in enumerate(real_columns):
        print(real_column)
        new_column_names = [f'{new_column_prefix}_{key}' if len(real_columns) == 1 else f'{new_column_prefix}_{index+1}_{key}' for key in info_names] 
        print(new_column_names)
            
        for new_column_name in new_column_names:
            new_columns_data[new_column_name] = []

        for movie_index, movie in df.iterrows():
            value = movie[real_column]
            str_date = str(get_date_from_movie(movie))
            if pd.isna(value):
                for col in new_column_names:
                    new_columns_data[col].append(np.nan)
            else:
                for name_index, new_column_name in enumerate(new_column_names):
                    year_info = info_dict[value][info_names[name_index]].get(str_date, np.nan)
                    new_columns_data[new_column_name].append(year_info)
    return new_columns_data

In [30]:
def add_features(df, 
                 columns,
                 new_column_prefix,
                 calc_movies_before=False, 
                 calc_budget=False, 
                 calc_experience=False,
                 calc_revenue=False):
    print('obtaine movies')
    
    t = time.process_time()
    data_info = get_info_df_movies_per_value_range_columns(df, columns)
    print(f'time: {time.process_time() - t}')
    
    data_info_copy = data_info.copy()
    print('calculate info')
    t = time.process_time()
    calculate_info_per_previous_movies_per_value(df, 
                                                 data_info_copy, 
                                                 calc_movies_before=calc_movies_before, 
                                                 calc_budget=calc_budget, 
                                                 calc_experience=calc_experience,
                                                 calc_revenue=calc_revenue)
    print(f'time: {time.process_time() - t}')
    
    print('get columns')
    t = time.process_time()
    feature_columns = get_columns_from_info_dict(df, data_info_copy, columns, new_column_prefix=new_column_prefix)
    print(f'time: {time.process_time() - t}')
    for column in feature_columns.keys():
        df[column] = feature_columns[column]
    return df


def add_companies_features(df):
    columns = get_real_columns('META__production_company_', '', 3)
    return add_features(df, columns, 'production_company', calc_revenue=True)


def add_cast_features(df):
    columns = get_real_columns('META__cast_', '_name', 8)
    return add_features(df, columns, 'cast', calc_movies_before=True, calc_revenue=True, calc_experience=True)


def add_crew_features(df):
    for column in [c for c in list(df.columns) if 'META__crew' in c]:
        df = add_features(df, [column], column.replace('META__', ''), calc_movies_before=True, calc_revenue=True)
    return df


def add_collection_features(df):
    return add_features(df, ['META__collection_name'], 'collection', calc_revenue=True)


def get_avg_cast_info(df):
    def get_average_cast_movies(row, columns):
        avgs = [row[col] for col in columns if not pd.isna(row[col])]
        return mean(avgs) if len(avgs) else np.nan
    
    def get_cast_average(df, column_name):
        columns = [col for col in df.columns if 'cast_' in col and column_name in col and not 'cast_avg' in col]
        return [get_average_cast_movies(row, columns) for i, row in df.iterrows()]
    
    for column in ['revenue', 'profit', 'experience', 'movies_before']:
        df[f'cast_avg_{column}'] = get_cast_average(df, column)
    return df


def prepare_df(df_original):
    overall_time = time.process_time()
    df = df_original.copy()
    df = add_companies_features(df)
    df = add_cast_features(df)
    df = add_crew_features(df)
    df = add_collection_features(df)
    df = get_avg_cast_info(df)
    df['year_avg_revenue'] = calc_smooth_mean(df, by='META__year', on='META__revenue', m=5)
    print(f'overall time: {time.process_time() - overall_time}')
    return df
    

In [33]:
raw_datasets = {
    'all': df_raw.copy(),
    'us': df_raw[
        df_raw.country__us == 1].copy(),
    'gb': df_raw[
        df_raw.country__gb == 1].copy(),
    'years2000-2018': df_raw[
        (df_raw.META__year >= 2000) &
        (df_raw.META__year <= 2018)].copy(),
    'years1970-1999': df_raw[
        (df_raw.META__year >= 1970) &
        (df_raw.META__year <= 1999)].copy(),
    'profitability_positive': df_raw[
        df_raw.META__profitability > 0].copy(),
    'profitability_negative': df_raw[
        df_raw.META__profitability < 0].copy(),
    'budget_start_1percent': df_raw[
        df_raw.budget >= 8875].copy(),
    'budget_start_5percent': df_raw[
        df_raw.budget >= 250000].copy(),
    'profitability_1percentile': df_raw[
        (df_raw.META__profitability >= -138) &
        (df_raw.META__profitability <= 65)].copy(),
    'profitability_5percentile': df_raw[
        (df_raw.META__profitability >= -14.9) &
        (df_raw.META__profitability <= 13.9)].copy(),
    'profitability_10percentile': df_raw[
        (df_raw.META__profitability >= -4.42) &
        (df_raw.META__profitability <= 7.4)].copy(),
    'profitability_1percentile_years2000-2018': df_raw[
        (df_raw.META__profitability >= -138) &
        (df_raw.META__profitability <= 65) &
        (df_raw.META__year >= 2000) &
        (df_raw.META__year <= 2018)].copy(),
    'profitability_1percentile_years2000-2018_budget_start_1percent': df_raw[
        (df_raw.META__profitability >= -138) &
        (df_raw.META__profitability <= 65) &
        (df_raw.META__year >= 2000) &
        (df_raw.META__year <= 2018) &
        (df_raw.budget >= 8875)].copy(),
    'profitability_1percentile_years2000-2018_budget_start_5percent': df_raw[
        (df_raw.META__profitability >= -138) &
        (df_raw.META__profitability <= 65) &
        (df_raw.META__year >= 2000) &
        (df_raw.META__year <= 2018) &
        (df_raw.budget >= 250000)].copy(),
    'revenue_1percentile': df_raw[
        (df_raw.META__revenue >= 10000) &
        (df_raw.META__revenue <= 854060072)].copy(),
    'revenue_5percentile': df_raw[
        (df_raw.META__revenue >= 111229) &
        (df_raw.META__revenue <= 348514752)].copy(),
}

In [34]:
for d in raw_datasets.keys():
    print(raw_datasets[d].shape[0])
for d in raw_datasets.keys():
    print(raw_datasets[d].shape[0]/df_raw.shape[0]*100)

7493
5695
893
4903
2042
5107
2335
7418
7129
7342
6744
5994
4814
4754
4599
7354
6743
100.0
76.00427065260911
11.917789937274788
65.43440544508208
27.252101961831045
68.1569464833845
31.162418257039903
98.99906579474175
95.14213265714668
97.98478580008008
90.00400373682103
79.99466168423862
64.24663018817563
63.445882823969036
61.37728546643534
98.1449352729214
89.99065794741759


In [35]:
for dataset in raw_datasets.keys():
    print('------------------------------------------------------')
    print(dataset)
    print('-------------------------------------------------------')
    raw_filename = f'datasets/dataset_{dataset}_raw.csv'
    filename = f'datasets/dataset_{dataset}.csv'
    raw_datasets[dataset].to_csv(raw_filename, index=False)
    raw_dataset = pd.read_csv(raw_filename)
    prepared = prepare_df(raw_dataset)
    prepared.to_csv(filename, index=False)

------------------------------------------------------
all
-------------------------------------------------------
obtaine movies
time: 32.76154373700001
calculate info
time: 51.697076400000014
get columns
['avg_profit', 'avg_revenue']
META__production_company_1
['production_company_1_avg_profit', 'production_company_1_avg_revenue']
META__production_company_2
['production_company_2_avg_profit', 'production_company_2_avg_revenue']
META__production_company_3
['production_company_3_avg_profit', 'production_company_3_avg_revenue']
time: 2.0308234389999598
obtaine movies
time: 106.78814797400003
calculate info
time: 80.79361381900003
get columns
['avg_profit', 'experience', 'avg_revenue', 'movies_before']
META__cast_1_name
['cast_1_avg_profit', 'cast_1_experience', 'cast_1_avg_revenue', 'cast_1_movies_before']
META__cast_2_name
['cast_2_avg_profit', 'cast_2_experience', 'cast_2_avg_revenue', 'cast_2_movies_before']
META__cast_3_name
['cast_3_avg_profit', 'cast_3_experience', 'cast_3_avg_rev

time: 0.6774820320001709
obtaine movies
time: 3.147450641999967
calculate info
time: 2.16235450399995
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__property_master
['crew__art__property_master_avg_profit', 'crew__art__property_master_avg_revenue', 'crew__art__property_master_movies_before']
time: 0.6782849279998118
obtaine movies
time: 7.0461755700000595
calculate info
time: 4.830708471999969
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__set_decoration
['crew__art__set_decoration_avg_profit', 'crew__art__set_decoration_avg_revenue', 'crew__art__set_decoration_movies_before']
time: 0.6725467409999055
obtaine movies
time: 2.8668473679999806
calculate info
time: 2.0953345089999402
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__visualeffects__visual_effects_supervisor
['crew__visualeffects__visual_effects_supervisor_avg_profit', 'crew__visualeffects__visual_effects_supervisor_avg_revenue', 'crew__visualeffe

time: 0.5004339019999406
obtaine movies
time: 9.560884298000019
calculate info
time: 5.448489851000204
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__costume__costume_designer
['crew__costume__costume_designer_avg_profit', 'crew__costume__costume_designer_avg_revenue', 'crew__costume__costume_designer_movies_before']
time: 0.5096633269999984
obtaine movies
time: 2.8845739339999454
calculate info
time: 1.721160736999991
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__costume__costume_supervisor
['crew__costume__costume_supervisor_avg_profit', 'crew__costume__costume_supervisor_avg_revenue', 'crew__costume__costume_supervisor_movies_before']
time: 0.5010722970000643
obtaine movies
time: 3.724021386000004
calculate info
time: 2.4718669090000276
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__costume__makeup_artist
['crew__costume__makeup_artist_avg_profit', 'crew__costume__makeup_artist_avg_revenue', 'crew__costume__mak

time: 1.2139902750000147
calculate info
time: 1.0431114590001016
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__directing__director__1
['crew__directing__director__1_avg_profit', 'crew__directing__director__1_avg_revenue', 'crew__directing__director__1_movies_before']
time: 0.07676021699990088
obtaine movies
time: 0.6961276120000548
calculate info
time: 0.4761828619998596
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__directing__script_supervisor
['crew__directing__script_supervisor_avg_profit', 'crew__directing__script_supervisor_avg_revenue', 'crew__directing__script_supervisor_movies_before']
time: 0.08008954000001722
obtaine movies
time: 1.5291948560000037
calculate info
time: 0.9028894409998429
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__production__casting
['crew__production__casting_avg_profit', 'crew__production__casting_avg_revenue', 'crew__production__casting_movies_before']
time: 0.07753322700000354
o

META__cast_5_name
['cast_5_avg_profit', 'cast_5_experience', 'cast_5_avg_revenue', 'cast_5_movies_before']
META__cast_6_name
['cast_6_avg_profit', 'cast_6_experience', 'cast_6_avg_revenue', 'cast_6_movies_before']
META__cast_7_name
['cast_7_avg_profit', 'cast_7_experience', 'cast_7_avg_revenue', 'cast_7_movies_before']
META__cast_8_name
['cast_8_avg_profit', 'cast_8_experience', 'cast_8_avg_revenue', 'cast_8_movies_before']
time: 3.4139326910001273
obtaine movies
time: 2.8075697539998146
calculate info
time: 1.7302352849999352
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__music_editor
['crew__sound__music_editor_avg_profit', 'crew__sound__music_editor_avg_revenue', 'crew__sound__music_editor_movies_before']
time: 0.42719095000006746
obtaine movies
time: 6.14535689000013
calculate info
time: 4.628648590000012
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__original_music_composer
['crew__sound__original_music_composer_avg_pro

...

META__cast_8_name
['cast_8_avg_profit', 'cast_8_experience', 'cast_8_avg_revenue', 'cast_8_movies_before']
time: 3.1905782440007897
obtaine movies
time: 2.711724394000157
calculate info
time: 1.6742312000005768
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__music_editor
['crew__sound__music_editor_avg_profit', 'crew__sound__music_editor_avg_revenue', 'crew__sound__music_editor_movies_before']
time: 0.4148681110000325
obtaine movies
time: 6.05908152900065
calculate info
time: 4.595228652000515
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__original_music_composer
['crew__sound__original_music_composer_avg_profit', 'crew__sound__original_music_composer_avg_revenue', 'crew__sound__original_music_composer_movies_before']
time: 0.4157394550002209
obtaine movies
time: 2.145218697999553
calculate info
time: 1.4467854229997101
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__sound_designer
['crew__sound_

time: 0.43334584399963205
obtaine movies
time: 1.9223838469997645
calculate info
time: 1.2152322090005327
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__camera__steadicam_operator
['crew__camera__steadicam_operator_avg_profit', 'crew__camera__steadicam_operator_avg_revenue', 'crew__camera__steadicam_operator_movies_before']
time: 0.4218123969994849
obtaine movies
time: 3.27607886000078
calculate info
time: 2.15976480000063
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__camera__still_photographer
['crew__camera__still_photographer_avg_profit', 'crew__camera__still_photographer_avg_revenue', 'crew__camera__still_photographer_movies_before']
time: 0.43207372300003044
obtaine movies
time: 1.3738515389995882
calculate info
time: 0.8865554410003824
get columns
['avg_profit', 'avg_revenue']
META__collection_name
['collection_avg_profit', 'collection_avg_revenue']
time: 0.4268673309998121
overall time: 355.1935167820002
--------------------------

time: 0.4002911829993536
obtaine movies
time: 3.277320881000378
calculate info
time: 3.1994917310003075
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__writing__screenplay__1
['crew__writing__screenplay__1_avg_profit', 'crew__writing__screenplay__1_avg_revenue', 'crew__writing__screenplay__1_movies_before']
time: 0.4037151000002268
obtaine movies
time: 3.6731306559995573
calculate info
time: 3.093828003999988
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__art_direction
['crew__art__art_direction_avg_profit', 'crew__art__art_direction_avg_revenue', 'crew__art__art_direction_movies_before']
time: 0.41124124599991774
obtaine movies
time: 6.430156292999527
calculate info
time: 4.39658052499999
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__art__production_design
['crew__art__production_design_avg_profit', 'crew__art__production_design_avg_revenue', 'crew__art__production_design_movies_before']
time: 0.4142729140003

time: 0.6639088739993895
obtaine movies
time: 10.943256979999205
calculate info
time: 8.877879609000047
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__production__producer__1
['crew__production__producer__1_avg_profit', 'crew__production__producer__1_avg_revenue', 'crew__production__producer__1_movies_before']
time: 0.6877513409999665
obtaine movies
time: 3.132661492999432
calculate info
time: 3.278778345000319
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__production__producer__2
['crew__production__producer__2_avg_profit', 'crew__production__producer__2_avg_revenue', 'crew__production__producer__2_movies_before']
time: 0.6781304080004702
obtaine movies
time: 12.488896402000137
calculate info
time: 8.564988083999197
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__editing__editor__1
['crew__editing__editor__1_avg_profit', 'crew__editing__editor__1_avg_revenue', 'crew__editing__editor__1_movies_before']
time: 0.65935

time: 0.5906714040002043
obtaine movies
time: 2.4609885469999426
calculate info
time: 1.617129852999824
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__sound_effects_editor
['crew__sound__sound_effects_editor_avg_profit', 'crew__sound__sound_effects_editor_avg_revenue', 'crew__sound__sound_effects_editor_movies_before']
time: 0.5857584079994922
obtaine movies
time: 3.8308347459997094
calculate info
time: 2.4788722690000213
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__sound_re_recording_mixer
['crew__sound__sound_re_recording_mixer_avg_profit', 'crew__sound__sound_re_recording_mixer_avg_revenue', 'crew__sound__sound_re_recording_mixer_movies_before']
time: 0.5898780450006598
obtaine movies
time: 3.3765117419998205
calculate info
time: 2.152590736000093
get columns
['avg_profit', 'avg_revenue', 'movies_before']
META__crew__sound__supervising_sound_editor
['crew__sound__supervising_sound_editor_avg_profit', 'crew__sound__super